Assignment - 

Use DVC or MLFLOW to convert this into a 
1. Create and API which will take the first input and it'll generate the next sentences
or 
2. Create and UI for this app

Submit it before Friday.

# Text generation with an RNN

In [9]:
import os
ROOT = "/content/drive/MyDrive/001_SUNNY_BHAVEEN_CHANDRA/001_My_Classes/FSDS_Feb_Batch/ALL_NOTES/May-21-22"
os.chdir(ROOT)

In [10]:
import tensorflow as tf
import numpy as np
import time

In [11]:
os.makedirs("data", exist_ok=True)

In [12]:
# !curl https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt > "./data/shakespeare.txt"

In [13]:
# samethinng can be done using config.yaml
class Config:
  path_to_file = os.path.join("data", "input.html")
  seq_length = 100

  Batch_size = 64
  Buffer_size = 10000 # 

  embedding_dim = 256

  rnn_units = 1024

  EPOCHS = 30

  checkpoint_dir = "./training_ckpt"

In [14]:
Config.EPOCHS

30

In [15]:
text = open(Config.path_to_file, "rb").read().decode(encoding='utf-8')
text[:1000]

'\n<!-- saved from url=(0039)https://en.wikipedia.org/wiki/Main_Page -->\n<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta name="color-scheme" content="light dark"></head><body><div class="line-gutter-backdrop"></div><form autocomplete="off"><label class="line-wrap-control">Line wrap<input type="checkbox" aria-label="Line wrap"></label></form><table><tbody><tr><td class="line-number" value="1"></td><td class="line-content"><span class="html-doctype">&lt;!DOCTYPE html&gt;</span></td></tr><tr><td class="line-number" value="2"></td><td class="line-content"><span class="html-tag">&lt;html <span class="html-attribute-name">class</span>="<span class="html-attribute-value">client-nojs</span>" <span class="html-attribute-name">lang</span>="<span class="html-attribute-value">en</span>" <span class="html-attribute-name">dir</span>="<span class="html-attribute-value">ltr</span>"&gt;</span></td></tr><tr><td class="line-number" value="3"></td><td class="line-cont

In [16]:
with open(Config.path_to_file) as f:
  t = f.read()

In [17]:
t[:100]

'\n<!-- saved from url=(0039)https://en.wikipedia.org/wiki/Main_Page -->\n<html><head><meta http-equiv='

In [18]:
vocab = sorted(set(text))
len(vocab)

188

In [19]:
char2idx = {uniChar: idx for idx, uniChar in enumerate(vocab)}
char2idx

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '#': 5,
 '$': 6,
 '%': 7,
 '&': 8,
 "'": 9,
 '(': 10,
 ')': 11,
 '*': 12,
 '+': 13,
 ',': 14,
 '-': 15,
 '.': 16,
 '/': 17,
 '0': 18,
 '1': 19,
 '2': 20,
 '3': 21,
 '4': 22,
 '5': 23,
 '6': 24,
 '7': 25,
 '8': 26,
 '9': 27,
 ':': 28,
 ';': 29,
 '<': 30,
 '=': 31,
 '>': 32,
 '?': 33,
 '@': 34,
 'A': 35,
 'B': 36,
 'C': 37,
 'D': 38,
 'E': 39,
 'F': 40,
 'G': 41,
 'H': 42,
 'I': 43,
 'J': 44,
 'K': 45,
 'L': 46,
 'M': 47,
 'N': 48,
 'O': 49,
 'P': 50,
 'Q': 51,
 'R': 52,
 'S': 53,
 'T': 54,
 'U': 55,
 'V': 56,
 'W': 57,
 'X': 58,
 'Y': 59,
 'Z': 60,
 '[': 61,
 '\\': 62,
 ']': 63,
 '_': 64,
 'a': 65,
 'b': 66,
 'c': 67,
 'd': 68,
 'e': 69,
 'f': 70,
 'g': 71,
 'h': 72,
 'i': 73,
 'j': 74,
 'k': 75,
 'l': 76,
 'm': 77,
 'n': 78,
 'o': 79,
 'p': 80,
 'q': 81,
 'r': 82,
 's': 83,
 't': 84,
 'u': 85,
 'v': 86,
 'w': 87,
 'x': 88,
 'y': 89,
 'z': 90,
 '{': 91,
 '|': 92,
 '}': 93,
 '®': 94,
 'à': 95,
 'â': 96,
 'å': 97,
 'ç': 98,
 'ê': 99,
 'ñ': 1

In [20]:
text_as_int = np.array([char2idx[c] for c in text])

text_as_int # all text is now represented as integer

array([ 1, 30,  3, ..., 77, 76, 32])

In [21]:
text[:13], text_as_int[:13]

('\n<!-- saved f', array([ 1, 30,  3, 15, 15,  2, 83, 65, 86, 69, 68,  2, 70]))

In [22]:
len(text)

404254

In [23]:
examples_per_epoch = len(text)//(Config.seq_length + 1)
examples_per_epoch

4002

In [24]:
idx2char_DICT = {val: key for key, val in char2idx.items()}

In [25]:
idx2char = np.array(vocab)
idx2char

array(['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*',
       '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7',
       '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D',
       'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q',
       'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
       '{', '|', '}', '®', 'à', 'â', 'å', 'ç', 'ê', 'ñ', 'ö', 'ú', 'ü',
       'ă', 'Č', 'č', 'š', 'ų', 'Ε', 'ά', 'η', 'ι', 'κ', 'λ', 'ν', 'Б',
       'М', 'Р', 'С', 'У', 'а', 'в', 'г', 'д', 'е', 'и', 'й', 'к', 'л',
       'н', 'о', 'п', 'р', 'с', 'т', 'у', 'х', 'ъ', 'ь', 'ї', 'ב', 'י',
       'ע', 'ר', 'ת', 'ا', 'ب', 'ة', 'ر', 'س', 'ع', 'ف', 'ل', 'ي', 'ی',
       'ং', 'ব', 'ল', 'া', 'ം', 'മ', 'യ', 'ല', 'ള', 'ാ', 'ท', 'ย', 'ไ',
       'ა', 'თ', 'ი', 'ლ', 'რ', 'უ', 'ქ', 'ế', 'ệ', '‐', '–',

In [26]:
idx2char[0], idx2char_DICT[0] 

('\t', '\t')

In [27]:
print("Hi\nThere")

Hi
There


In [28]:
print(r"Hi\nThere")

Hi\nThere


In [29]:
print(repr("Hi\nThere"))

'Hi\nThere'


In [30]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(13):
  print(idx2char[i.numpy()])



<
!
-
-
 
s
a
v
e
d
 
f


In [31]:
sequences = char_dataset.batch(Config.seq_length + 1, drop_remainder=True)

for item in sequences.take(10):
  to_print = repr("".join([idx2char[c] for c in item.numpy()]))
  print(to_print)
  print(len(to_print))

'\n<!-- saved from url=(0039)https://en.wikipedia.org/wiki/Main_Page -->\n<html><head><meta http-equiv="'
105
'Content-Type" content="text/html; charset=UTF-8"><meta name="color-scheme" content="light dark"></hea'
103
'd><body><div class="line-gutter-backdrop"></div><form autocomplete="off"><label class="line-wrap-cont'
103
'rol">Line wrap<input type="checkbox" aria-label="Line wrap"></label></form><table><tbody><tr><td clas'
103
's="line-number" value="1"></td><td class="line-content"><span class="html-doctype">&lt;!DOCTYPE html&'
103
'gt;</span></td></tr><tr><td class="line-number" value="2"></td><td class="line-content"><span class="'
103
'html-tag">&lt;html <span class="html-attribute-name">class</span>="<span class="html-attribute-value"'
103
'>client-nojs</span>" <span class="html-attribute-name">lang</span>="<span class="html-attribute-value'
103
'">en</span>" <span class="html-attribute-name">dir</span>="<span class="html-attribute-value">ltr</sp'
103
'an>"&gt;</span></td></tr>

In [33]:
# First Citize -> irst Citizen

In [34]:
def split_input_target(chunk):
  input_text = chunk[:-1] # First Citize
  target_text = chunk[1:] # irst Citizen
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [35]:
for input_example, target_example in dataset.take(1):
  print("input_data:\n")
  print(repr("".join([idx2char[i] for i in input_example.numpy()])))
  print("\n\ntarget_data:\n")
  print(repr("".join([idx2char[t] for t in target_example.numpy()])))

input_data:

'\n<!-- saved from url=(0039)https://en.wikipedia.org/wiki/Main_Page -->\n<html><head><meta http-equiv='


target_data:

'<!-- saved from url=(0039)https://en.wikipedia.org/wiki/Main_Page -->\n<html><head><meta http-equiv="'


In [36]:
dataset = dataset.shuffle(Config.Buffer_size).batch(Config.Batch_size, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [37]:
Config.vocab_size = len(vocab)
Config.vocab_size

188

In [38]:
def build_model(
    vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
          tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
          tf.keras.layers.GRU(rnn_units,
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform'),
          tf.keras.layers.Dense(vocab_size)
  ])
  return model


In [39]:
model = build_model(
    vocab_size = Config.vocab_size,
    embedding_dim = Config.embedding_dim,
    rnn_units = Config.rnn_units,
    batch_size = Config.Batch_size
)

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           48128     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 188)           192700    
                                                                 
Total params: 4,179,132
Trainable params: 4,179,132
Non-trainable params: 0
_________________________________________________________________


In [41]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [42]:
model.compile(optimizer='adam', loss=loss)

In [43]:
checkpoint_prefix = os.path.join(Config.checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

In [44]:
history = model.fit(dataset, epochs=Config.EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
62/62 [==============================] - 20s 198ms/step - loss: 2.7527
Epoch 2/30
62/62 [==============================] - 13s 196ms/step - loss: 1.0557
Epoch 3/30
62/62 [==============================] - 13s 195ms/step - loss: 0.6784
Epoch 4/30
62/62 [==============================] - 13s 196ms/step - loss: 0.5568
Epoch 5/30
62/62 [==============================] - 13s 193ms/step - loss: 0.4859
Epoch 6/30
62/62 [==============================] - 13s 196ms/step - loss: 0.4327
Epoch 7/30
62/62 [==============================] - 13s 195ms/step - loss: 0.3895
Epoch 8/30
62/62 [==============================] - 13s 195ms/step - loss: 0.3536
Epoch 9/30
62/62 [==============================] - 13s 197ms/step - loss: 0.3245
Epoch 10/30
62/62 [==============================] - 13s 197ms/step - loss: 0.2979
Epoch 11/30
62/62 [==============================] - 13s 198ms/step - loss: 0.2767
Epoch 12/30
62/62 [==============================] - 13s 196ms/step - loss: 0.2569
Epoch 13/30
6

In [45]:
(len(text)/Config.Batch_size)/(Config.seq_length + 1)

62.539294554455445

In [46]:
# Restoring checkoint - 

tf.train.latest_checkpoint(Config.checkpoint_dir)

'./training_ckpt/ckpt_30'

In [47]:
model_from_ckpt = build_model(
    vocab_size = Config.vocab_size,
    embedding_dim = Config.embedding_dim,
    rnn_units = Config.rnn_units,
    batch_size = 1
)

In [48]:
model_from_ckpt.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            48128     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 188)            192700    
                                                                 
Total params: 4,179,132
Trainable params: 4,179,132
Non-trainable params: 0
_________________________________________________________________


In [49]:
model_from_ckpt.load_weights(tf.train.latest_checkpoint(Config.checkpoint_dir))

model_from_ckpt.build(tf.TensorShape([1, None]))

In [50]:
model_from_ckpt.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            48128     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 188)            192700    
                                                                 
Total params: 4,179,132
Trainable params: 4,179,132
Non-trainable params: 0
_________________________________________________________________


# prediction

In [51]:
# define a 
def generate_text(model, start_string, no_of_chars_to_gen=1000):
  # convert the input text to nos.
  input_val = [char2idx[s] for s in start_string] # text converted to int
  input_val = tf.expand_dims(input_val, 0) # [] ->> [1, ]

  text_generated = list()

  temperature = 1.0


  model.reset_states() # Resetting the previous states if any while predictions.
  for i in range(no_of_chars_to_gen):
    predictions = model(input_val)

    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature
    # print(predictions)
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
    # print(predicted_id)
    
    input_val = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return start_string + "".join(text_generated)

In [68]:
result = generate_text(model=model_from_ckpt, start_string="<body>",
                      no_of_chars_to_gen=1500)
print(result)

<body>"html-attribute-value">Nor-to contact Wikipedia.wikimedia.org/w/index.php?title=Special:DownloadAsPrf&act;</span><span class="html-tag">&lt;meta <span class="html-attribute-name">class</span>="<span class="html-attribute-value">Sloven&amp;#160;marsian th class="html-attribute-value">interlanguage-link-target</span>"&gt;</span><span class="html-tag">&lt;a <span class="html-attribute-name">class</span>="<span class="html-attribute-value">100 Wok:mw-Breenene-value html-external-link" target="_blank" href="https://upload.wikimedia.org/wikipedia/commons/thumb/7/75/Wikinews-logo.svg/100px-Wikinews-logo.svg.png" rel="noreferrer noopener">//upload.wikimedia.org/wikipedia/commons/thumb/b/ba/Inna_Derusova_with_Defender_of_the_Motherland_Medal.jpg</a>" <span class="html-attribute-name">stype</span>="<span class="html-attribute-value">async</span>" <span class="html-attribute-name">class</span>="<span class="html-attribute-value">410</span>" <span class="html-attribute-name">href</span>="<a 

In [69]:
# !pip install HTMLrenderer

In [70]:
from HTMLrenderer.render import render_HTML
render_HTML(result)

In [53]:
tf.math.log([[0.5, 0.5]])

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.6931472, -0.6931472]], dtype=float32)>

In [54]:

for _ in range(15):
  predictions = [[1000., 1.,2.,3.,4., 55., 56., 100., 101., 200., 1001]]
  samples = tf.random.categorical(predictions, 1)[-1, 0]
  print(samples.numpy())
  # print(values[samples.numpy()])

10
10
0
10
10
10
10
10
10
10
0
10
0
10
10
